In [13]:
from pymongo import MongoClient
import json
from collections import Counter
import glob
import os
from pathlib import Path

In [14]:
images_dir = Path('./images')

In [2]:
mongo = MongoClient('172.17.0.1', 27017)
db = mongo['bad-vis']
posts = db['posts']

## Valid image URLs

In [3]:
all_posts = [p for p in posts.find()]

In [4]:
source_url = {'https://www.reddit.com/r/dataisugly', 'https://badvisualisations.tumblr.com/', 'https://viz.wtf/'}

In [5]:
output = [p for p in all_posts
              if p['source_url'] in source_url and
                 p['preview']['url'] != '' and
                 p['thumbnail']['url'] != '' and
                 p['media_type'] == 'image'
         ]
len(output) # expected to have valid images

6600

In [6]:
results = [r for r in db['dlarchiveresults'].find() if 'status' in r and (r['status'] == 'exists' or r['status'] == 'downloaded')]
len(results)

0

In [7]:
results = [r for r in db['dlarchiveresults'].find() if 'status' in r and r['status'] == 'skip_content_type']
len(results)

0

In [8]:
results = [r for r in db['dlarchiveresults'].find() if 'archive_status' in r and r['archive_status'] == 'archive_not_found']
len(results)

0

In [9]:
Counter([r['status'] for r in db['dlarchiveresults'].find() if 'status' in r])

Counter()

In [25]:
%%time
posts_with_image = [p for p in posts.find() if glob.glob(f"images/*/*/*/{p['id']}*")]

CPU times: user 2min 12s, sys: 1min 2s, total: 3min 15s
Wall time: 3min 15s


In [27]:
len(posts_with_image)

7852

In [30]:
len(all_posts) - len(posts_with_image)

918

In [31]:
for p in [posts.find_one()]:
    print(p['id'])
    print(glob.glob(f"images/*/*/*/{p['id']}*"))

66i283
['images/preview/reddit/dataisugly/66i283.jpg', 'images/thumbnail/reddit/dataisugly/66i283.jpg', 'images/external_link/reddit/dataisugly/66i283.jpg']


In [32]:
Counter([p['id'] for p in all_posts]).most_common()[:10]

[('66i283', 1),
 ('e2cvur', 1),
 ('6h6cr9', 1),
 ('4xjubo', 1),
 ('boo6ld', 1),
 ('6wqste', 1),
 ('8msftx', 1),
 ('5a0wp5', 1),
 ('a3gj0h', 1),
 ('bzk2hf', 1)]

In [20]:
Counter([os.path.splitext(name)[1] for _, _, files in os.walk(images_dir) for name in files])

Counter({'.jpg': 11071, '.png': 7728, '.gif': 128})

# Download results

In [ ]:
# df_dl_preview_results = pd.DataFrame.from_records(dl_preview_results)
# df_dl_preview_results.loc[(df_dl_preview_results['status'] != 'exists') &
#                           (df_dl_preview_results['status'] != 'invalid_url') &
#                           (~df_dl_preview_results['status'].isnull()) &
#                           (df_dl_preview_results['status'] != 'downloaded')].to_csv('./dl_preview_results.csv', index=False)

In [ ]:
# df_dl_thumbnail_results = pd.DataFrame.from_records(dl_thumbnail_results)
# df_dl_thumbnail_results.loc[(df_dl_thumbnail_results['status'] != 'exists') &
#                             (df_dl_thumbnail_results['status'] != 'invalid_url') &
#                             (~df_dl_thumbnail_results['status'].isnull()) &
#                             (df_dl_thumbnail_results['status'] != 'downloaded')].to_csv('./dl_thumbnail_results.csv', index=False)

In [ ]:
# df_dl_external_link_results = pd.DataFrame.from_records(dl_external_link_results)
# df_dl_external_link_results.loc[(df_dl_external_link_results['status'] != 'exists') &
#                           (df_dl_external_link_results['status'] != 'invalid_url') &
# #                           (df_dl_external_link_results['status'] != 'not_found') &
# #                           (df_dl_external_link_results['status'] != 'http_error') &
#                           (~df_dl_external_link_results['status'].isnull()) &
#                           (~df_dl_external_link_results['status'].str.contains('skip_')) &
#                           (df_dl_external_link_results['status'] != 'downloaded')].to_csv('./dl_external_link_results.csv', index=False)

## Tags

In [24]:
c = Counter([t for p in posts.find() for t in p['tags']])

In [25]:
c.most_common()[:50]

[('WTFViz', 2119),
 ('Percentages', 985),
 ('BarChart', 705),
 ('submission', 704),
 ('PieChart', 477),
 ('DonutChart', 373),
 ('3D', 304),
 ('Radial', 276),
 ('Bad Scale', 231),
 ('Diagram', 162),
 ('Map', 156),
 ('Pie Gore', 150),
 ('LabelLines', 147),
 ('Clusterfuck', 139),
 ('LineChart', 131),
 ('OC', 113),
 ('What the Fuck?', 94),
 ('Iconography', 88),
 ('StackedBarChart', 84),
 ('Agendas Gone Wild', 81),
 ('Timeline', 81),
 ('Wedges', 70),
 ('Arrows', 69),
 ('BubbleChart', 67),
 ('CoxcombChart', 63),
 ('Euler', 61),
 ('R2: Nothing wrong', 55),
 ('data visualisation', 50),
 ('AreaChart', 45),
 ('data visualization', 42),
 ('NotPartToWhole', 35),
 ('Venn', 32),
 ('R1: Parody', 31),
 ('bar graph', 29),
 ('Rainbow', 28),
 ('Area/Volume', 27),
 ('Clock', 26),
 ('3D is Cool!', 25),
 ('NotPartWhole', 25),
 ('R3: Dataisbeautiful', 20),
 ('Flawed Flows', 19),
 ('PartToWhole', 19),
 ('Scatterplot', 18),
 ('NetworkDiagram', 16),
 ('NotPeriodic', 16),
 ('TriangleChart', 15),
 ('Dial', 15),
 

## Author

In [26]:
c = Counter([p['author'] for p in posts.find() if 'author' in p])

In [27]:
c.most_common()[:50]

[('', 1374),
 ('rhiever', 80),
 ('CJMinard', 58),
 ('minimaxir', 36),
 ('zonination', 34),
 ('[deleted]', 31),
 ('akeemtheafricandream', 28),
 ('NelsonMinar', 24),
 ('jensyao', 20),
 ('confused_confucius07', 19),
 ('ProgVal', 17),
 ('OllyTwist', 16),
 ('Renegade_Meister', 15),
 ('zevobh', 14),
 ('denacioust', 14),
 ('Epistaxis', 12),
 ('almodozo', 11),
 ('misnamed', 11),
 ('suity1', 10),
 ('WE_THEPEOPLE', 10),
 ('phil_blumpkin', 10),
 ('wspaniel', 10),
 ('SymphMeta', 10),
 ('iqueerified', 9),
 ('WHY_DO_I_SHOUT', 8),
 ('crvc', 8),
 ('Thefriendlyfaceplant', 8),
 ('MatthewLaw', 8),
 ('Batrachus', 8),
 ('colako', 8),
 ('pongpingpongpingpong', 7),
 ('Catharrrsis', 7),
 ('DiamondMinah', 7),
 ('nate_silver_surfer', 7),
 ('Curran919', 7),
 ('toot_ma_boot', 7),
 ('Null_HHockey', 7),
 ('mrdelayer', 7),
 ('ar_604', 6),
 ('wheresflateric', 6),
 ('ofsinope', 6),
 ('Reddits_Worst_Night', 6),
 ('rocketman0739', 6),
 ('--DrScience--', 6),
 ('bubbles212', 6),
 ('mrgann', 6),
 ('owmyfoothurts', 6),
 ('v